# LLM Conversion

<link rel="stylesheet" href="/mediapipe/site.css">

<table align="left" class="buttons">
  <td>
    <a href="https://colab.research.google.com/github/googlesamples/mediapipe/blob/main/examples/llm_inference/conversion/llm_conversion.ipynb" target="_blank">
      <img src="https://developers.google.com/static/mediapipe/solutions/customization/colab-logo-32px_1920.png" alt="Colab logo"> Run in Colab
    </a>
  </td>

  <td>
    <a href="https://github.com/googlesamples/mediapipe/blob/main/examples/llm_inference/conversion/llm_conversion.ipynb" target="_blank">
      <img src="https://developers.google.com/static/mediapipe/solutions/customization/github-logo-32px_1920.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

In [1]:
#@title License information { display-mode: "form" }
#@markdown Copyright 2024 The MediaPipe Authors.
#@markdown Licensed under the Apache License, Version 2.0 (the "License");
#@markdown
#@markdown you may not use this file except in compliance with the License.
#@markdown You may obtain a copy of the License at
#@markdown
#@markdown https://www.apache.org/licenses/LICENSE-2.0
#@markdown
#@markdown Unless required by applicable law or agreed to in writing, software
#@markdown distributed under the License is distributed on an "AS IS" BASIS,
#@markdown WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#@markdown See the License for the specific language governing permissions and
#@markdown limitations under the License.

In [2]:
# @title Setup { display-mode: "form" }
# @markdown import ipywidgets\
# @markdown import IPython.display\
# @markdown import os\
# @markdown import huggingface downloader\
# @markdown import mediapipe genai converter
import ipywidgets as widgets
from IPython.display import display
install_out = widgets.Output()
display(install_out)
with install_out:
  !pip install mediapipe
  !pip install torch
  !pip install huggingface_hub
  import os
  from huggingface_hub import hf_hub_download
  from mediapipe.tasks.python.genai import converter

install_out.clear_output()
with install_out:
  print("Setup done.")


Output()

In [3]:
# @title Run { display-mode: "form"}

model = widgets.Dropdown(
    options=["Gemma 2B", "Falcon 1B", "StableLM 3B", "Phi 2"],
    value='Gemma 2B',
    description='model',
    disabled=False,
)

backend = widgets.Dropdown(
    options=["cpu", "gpu"],
    value='cpu',
    description='backend',
    disabled=False,
)

token = widgets.Password(
    value='',
    placeholder='huggingface token',
    description='HF token:',
    disabled=False
)

def on_change_model(change):
  if change["new"] != 'Gemma 2b':
    token_description.layout.display = "none"
    token.layout.display = "none"
  else:
    token_description.layout.display = "flex"
    token.layout.display = "flex"

model.observe(on_change_model, names=['value'])



display(model)
display(backend)

token_description = widgets.Output()
with token_description:
  print("Huggingface token needed for gated model (e.g. Gemma)")
  print("You can get it from https://huggingface.co/settings/tokens")

display(token_description)
display(token)



def gemma_download(token):
  REPO_ID = "google/gemma-2b-it"
  FILENAMES = ["tokenizer.json", "tokenizer_config.json", "model-00001-of-00002.safetensors", "model-00002-of-00002.safetensors"]
  os.environ['HF_TOKEN'] = token
  with out:
    for filename in FILENAMES:
      hf_hub_download(repo_id=REPO_ID, filename=filename, local_dir="./gemma-2b-it")

def falcon_download():
  REPO_ID = "tiiuae/falcon-rw-1b"
  FILENAMES = ["tokenizer.json", "tokenizer_config.json", "pytorch_model.bin"]
  with out:
    for filename in FILENAMES:
      hf_hub_download(repo_id=REPO_ID, filename=filename, local_dir="./falcon-rw-1b")

def stablelm_download():
  REPO_ID = "stabilityai/stablelm-3b-4e1t"
  FILENAMES = ["tokenizer.json", "tokenizer_config.json", "model.safetensors"]
  with out:
    for filename in FILENAMES:
      hf_hub_download(repo_id=REPO_ID, filename=filename, local_dir="./stablelm-3b-4e1t")

def phi2_download():
  REPO_ID = "microsoft/phi-2"
  FILENAMES = ["tokenizer.json", "tokenizer_config.json", "model-00001-of-00002.safetensors", "model-00002-of-00002.safetensors"]
  with out:
    for filename in FILENAMES:
      hf_hub_download(repo_id=REPO_ID, filename=filename, local_dir="./phi-2")


def gemma_convert_config(backend):
  input_ckpt = '/content/gemma-2b-it/'
  vocab_model_file = '/content/gemma-2b-it/'
  output_dir = '/content/intermediate/gemma-2b-it/'
  output_tflite_file = f'/content/converted_models/gemma_{backend}.bin'
  return converter.ConversionConfig(input_ckpt=input_ckpt, ckpt_format='safetensors', model_type='GEMMA_2B', backend=backend, output_dir=output_dir, combine_file_only=False, vocab_model_file=vocab_model_file, output_tflite_file=output_tflite_file)

def falcon_convert_config(backend):
  input_ckpt = '/content/falcon-rw-1b/pytorch_model.bin'
  vocab_model_file = '/content/falcon-rw-1b/'
  output_dir = '/content/intermediate/falcon-rw-1b/'
  output_tflite_file = f'/content/converted_models/falcon_{backend}.bin'
  return converter.ConversionConfig(input_ckpt=input_ckpt, ckpt_format='pytorch', model_type='FALCON_RW_1B', backend=backend, output_dir=output_dir, combine_file_only=False, vocab_model_file=vocab_model_file, output_tflite_file=output_tflite_file)

def stablelm_convert_config(backend):
  input_ckpt = '/content/stablelm-3b-4e1t/'
  vocab_model_file = '/content/stablelm-3b-4e1t/'
  output_dir = '/content/intermediate/stablelm-3b-4e1t/'
  output_tflite_file = f'/content/converted_models/stablelm_{backend}.bin'
  return converter.ConversionConfig(input_ckpt=input_ckpt, ckpt_format='safetensors', model_type='STABLELM_4E1T_3B', backend=backend, output_dir=output_dir, combine_file_only=False, vocab_model_file=vocab_model_file, output_tflite_file=output_tflite_file)


def phi2_convert_config(backend):
  input_ckpt = '/content/phi-2'
  vocab_model_file = '/content/phi-2/'
  output_dir = '/content/intermediate/phi-2/'
  output_tflite_file = f'/content/converted_models/phi2_{backend}.bin'

  return converter.ConversionConfig(input_ckpt=input_ckpt, ckpt_format='safetensors', model_type='PHI_2', backend=backend, output_dir=output_dir, combine_file_only=False, vocab_model_file=vocab_model_file, output_tflite_file=output_tflite_file)


def on_button_clicked(b):
  try:
    out.clear_output()
    with out:
      print("Downloading model ...")
    button.description = "Downloading ..."
    button.disabled = True
    model.disabled = True
    backend.disabled = True

    if model.value == 'Gemma 2B':
      gemma_download(token.value)
    elif model.value == 'Falcon 1B':
      falcon_download()
    elif model.value == 'StableLM 3B':
      stablelm_download()
    elif model.value == 'Phi 2':
      phi2_download()
    else:
      raise Exception("Invalid model")

    with out:
      print("Done")
      print("Converting model ...")

    button.description = "Converting ..."

    if model.value == 'Gemma 2B':
      config = gemma_convert_config(backend.value)
    elif model.value == 'Falcon 1B':
      config = falcon_convert_config(backend.value)
    elif model.value == 'StableLM 3B':
      config = stablelm_convert_config(backend.value)
    elif model.value == 'Phi 2':
      config = phi2_convert_config(backend.value)
    else:
      with out:
        raise Exception("Invalid model")
      return

    with out:
      converter.convert_checkpoint(config)
      print("Done")

    button.description = "Start Conversion"
    button.disabled = False
    model.disabled = False
    backend.disabled = False

  except Exception as e:
    button.description = "Start Conversion"
    button.disabled = False
    model.disabled = False
    backend.disabled = False
    with out:
      print(e)


button = widgets.Button(description="Start Conversion")

button.on_click(on_button_clicked)
display(button)

out = widgets.Output(layout={'border': '1px solid black'})
display(out)

print("\nNotice: Converted models are saved under ./converted_models")

Dropdown(description='model', options=('Gemma 2B', 'Falcon 1B', 'StableLM 3B', 'Phi 2'), value='Gemma 2B')

Dropdown(description='backend', options=('cpu', 'gpu'), value='cpu')

Output()

Password(description='HF token:', placeholder='huggingface token')

Button(description='Start Conversion', style=ButtonStyle())

Output(layout=Layout(border='1px solid black'))


Notice: Converted models are saved under ./converted_models


In [7]:
# @title Save to google drive { display-mode: "form"}
google_drive_directory = "converted_models" #@param {type:"string"}
from google.colab import drive
drive.mount('/content/drive')
print("Copying models ...")
!mkdir -p /content/drive/MyDrive/$google_drive_directory
!cp -r -f /content/converted_models/* /content/drive/MyDrive/$google_drive_directory
print("Done")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Copying models ...
Done
